In [ ]:
#| default_exp api_clients

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import httpx
from typing import Dict, Any, List
import re
import itertools
import threading


In [ ]:
#| export
# Thread-safe round-robin between two Polaris APIs
_polaris_urls = ["http://localhost:8001/paraphrase", "http://localhost:8003/paraphrase"]
_polaris_index = itertools.count()
_polaris_lock = threading.Lock()

def _get_next_polaris_url() -> str:
    """Get next Polaris URL in thread-safe round-robin fashion."""
    with _polaris_lock:
        idx = next(_polaris_index) % len(_polaris_urls)
    return _polaris_urls[idx]


In [ ]:
#| export
def call_polaris_api(
    prescription_id: str,
    pharmacy_name: str,
    pharmacy_code: str,
    prescription_date: str,
    patient_age: int,
    patient_gender: str,
    prescription_items: List[Dict],
    model: str = "polaris:latest"
) -> dict:
    """Call Polaris API to get target_direction for prescription items."""
    url = _get_next_polaris_url()
    
    prescription_data = {
        "prescription_id": prescription_id,
        "pharmacy_name": pharmacy_name,
        "pharmacy_code": pharmacy_code,
        "prescription_date": prescription_date,
        "patient_age": patient_age,
        "patient_gender": patient_gender,
        "prescription_items": prescription_items
    }
    
    payload = {
        "prescription": prescription_data,
        "config": {"model": model}
    }
    
    response = httpx.post(url, json=payload, timeout=120)
    response.raise_for_status()
    return response.json()


In [ ]:
#| export
def call_parser_api(
    prescription_data: Dict[str, Any]
) -> dict:
    """Call Parser API to get dosage_fields for prescription items."""
    url = "http://localhost:8002/parse-prescription"
    
    response = httpx.post(url, json=prescription_data, timeout=120)
    response.raise_for_status()
    return response.json()


In [ ]:
#| export
def call_polaris_api_bulk(
    prescriptions_data: List[Dict[str, Any]],
    model: str = "polaris:latest"
) -> List[dict]:
    """Call Polaris bulk API to process multiple prescriptions at once."""
    url = _get_next_polaris_url()  # Use the function instead of next()
    
    payload = {
        "prescriptions": prescriptions_data,
        "config": {"model": model}
    }
    
    response = httpx.post(url.replace('/paraphrase', '/paraphrase/bulk'), json=payload, timeout=300)
    response.raise_for_status()
    return response.json()['results']


In [ ]:
#| export
def extract_target_direction(text: str) -> str:
    """Extract clean target_direction from XML-formatted Polaris response."""
    if not text:
        return text
    
    # Try to extract from <target_direction> tags
    match = re.search(r'<target_direction>(.*?)</target_direction>', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # If no tags found, return original text
    return text
